In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# ระบุ path ของโฟลเดอร์ที่เก็บไฟล์
input_folder = r"C:\Users\BMEI CMU\Documents\G6PD\Dataset"
output_folder = r"C:\Users\BMEI CMU\Documents\G6PD\Cropped_Objects"

# ตรวจสอบว่าโฟลเดอร์ output มีอยู่หรือไม่ ถ้าไม่มีก็สร้างขึ้นมา
os.makedirs(output_folder, exist_ok=True)

# ฟังก์ชันสำหรับการแสดง Threshold ต่างๆ
def find_object_with_threshold(original_image, gray_image, threshold_value):
    # ใช้ Threshold เพื่อแยก object
    _, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)
    
    # หา Contours ของ object
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # ตรวจสอบว่ามีวัตถุหรือไม่
    if len(contours) == 0:
        return None

    # วาด Contours และหาส่วนที่ใหญ่ที่สุด
    largest_contour = max(contours, key=cv2.contourArea)  # หา contour ที่มีขนาดใหญ่ที่สุด
    x, y, w, h = cv2.boundingRect(largest_contour)  # หาพื้นที่สี่เหลี่ยมที่ล้อมรอบ object

    # Crop เฉพาะส่วน object จากภาพต้นฉบับ
    cropped_object = original_image[y:y+h, x:x+w]
    
    return cropped_object, x, y, w, h

# วนลูปผ่านไฟล์ทั้งหมดในโฟลเดอร์
for file_name in os.listdir(input_folder):
    if file_name.endswith(".png") and ("-5" in file_name or "-15" in file_name):  # เลือกเฉพาะไฟล์ที่ตรงตามเงื่อนไข
        file_path = os.path.join(input_folder, file_name)

        # โหลดรูปภาพ
        image = cv2.imread(file_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # แปลงเป็นภาพสีเทา

        # ทดลองค่า Threshold
        threshold_value = 5
        result = find_object_with_threshold(image, gray_image, threshold_value)

        if result is not None:
            cropped_object, x, y, w, h = result
            
            # บันทึกไฟล์ที่ crop
            save_path = os.path.join(output_folder, file_name)
            cv2.imwrite(save_path, cropped_object)
            #print(f"บันทึกไฟล์: {save_path}")
        else:
            #print(f"ไม่พบวัตถุในไฟล์: {file_name} ที่ Threshold: {threshold_value}")


In [ ]:
import cv2
import numpy as np
import os

# ระบุ path ของโฟลเดอร์ภาพ
input_folder = r"C:\Users\BMEI CMU\Documents\G6PD\Cropped_Objects"
output_folder = r"C:\Users\BMEI CMU\Documents\G6PD\Split_Images"

# สร้างโฟลเดอร์สำหรับบันทึกผลลัพธ์ (ถ้ายังไม่มี)
os.makedirs(output_folder, exist_ok=True)

# วนลูปอ่านไฟล์ในโฟลเดอร์
for file_name in os.listdir(input_folder):
    if file_name.endswith(".png") and ("-5" in file_name or "-15" in file_name):  # ตรวจสอบชื่อไฟล์
        file_path = os.path.join(input_folder, file_name)

        # โหลดภาพต้นฉบับแบบสี (BGR)
        image_color = cv2.imread(file_path, cv2.IMREAD_COLOR)
        image_rgb = cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB)  # แปลงเป็น RGB

        # แปลงภาพเป็น Grayscale
        image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)

        # แบ่งภาพเป็น 3 ส่วนในแนวตั้ง
        height, width = image_gray.shape
        part_width = width // 3
        middle_section = image_gray[:, part_width:2 * part_width]  # ส่วนตรงกลาง

        # คำนวณค่าเฉลี่ยความเข้มแสงของพื้นที่ตรงกลาง
        middle_intensity = np.mean(middle_section, axis=0)

        # หาตำแหน่งที่ค่าความเข้มแสงต่ำที่สุดในส่วนกลาง
        middle_black_x_relative = np.argmin(middle_intensity)

        # คำนวณตำแหน่งที่แท้จริงในภาพต้นฉบับ
        middle_black_x = middle_black_x_relative + part_width

        # ตรวจสอบความถูกต้อง: หากค่าต่ำสุดไม่สมเหตุสมผล ให้ใช้การแบ่งครึ่งแทน
        if middle_black_x <= 0 or middle_black_x >= width:
            middle_black_x = width // 2

        # แบ่งภาพต้นฉบับออกเป็นซ้ายและขวา
        left_half = image_rgb[:, :middle_black_x, :]  # ส่วนซ้าย
        right_half = image_rgb[:, middle_black_x:, :]  # ส่วนขวา

        # บันทึกภาพซ้ายและขวา
        left_file_name = f"{os.path.splitext(file_name)[0]}_left.png"
        right_file_name = f"{os.path.splitext(file_name)[0]}_right.png"
        cv2.imwrite(os.path.join(output_folder, left_file_name), cv2.cvtColor(left_half, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(output_folder, right_file_name), cv2.cvtColor(right_half, cv2.COLOR_RGB2BGR))

        print(f"บันทึก: {left_file_name} และ {right_file_name} เรียบร้อย")

# สรุปผลลัพธ์
print(f"การแบ่งภาพเสร็จสมบูรณ์! ผลลัพธ์ถูกบันทึกในโฟลเดอร์: {output_folder}")

In [54]:
import cv2
import numpy as np
import os
import glob

def adaptive_threshold(img, min_intensity=None, max_intensity=None, adjustment_factor=1.0):
    if min_intensity is None or max_intensity is None:
        idx = img[img != 0]
        max_intensity = np.max(idx)
        min_intensity = np.min(idx)
    
    # ปรับค่า max_intensity และ min_intensity ตาม adjustment_factor
    max_intensity *= adjustment_factor
    min_intensity *= adjustment_factor

    r = min_intensity + (max_intensity - min_intensity) * np.random.rand()
    T = r
    newT = 9999
    k = 0

    while k != 500:
        G1 = img[img <= T]
        nG1 = np.sum(G1 != 0)
        M1 = np.sum(G1) / nG1 if nG1 > 0 else 0
        
        G2 = img[img > T]
        M2 = np.sum(G2) / len(G2) if len(G2) > 0 else 0
        
        if np.isnan(M2):
            M2 = 0
        
        newT = (M1 + M2) / 2
        
        if T == newT:
            break
        else:
            T = newT
        
        k += 1
    
    return T

# Paths
base_path = r'C:\Users\BMEI CMU\Documents\G6PD\Split_Images'
output_path = r'C:\Users\BMEI CMU\Documents\G6PD\Processed_Images'

# Create output folder
os.makedirs(output_path, exist_ok=True)

# Find files
patterns = ['*-5_left.png', '*-5_right.png', '*-15_left.png', '*-15_right.png']
file_paths = []

for pattern in patterns:
    file_paths += glob.glob(os.path.join(base_path, pattern))

if not file_paths:
    print("ไม่พบไฟล์ที่ตรงกับรูปแบบที่กำหนด")
else:
    print(f"พบ {len(file_paths)} ไฟล์ที่ตรงกับรูปแบบ")

# Process files
for file_path in file_paths:
    image = cv2.imread(file_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    # Adjust parameters based on file type (left or right)
    filename = os.path.basename(file_path)
    if "left" in filename:
        adjustment_factor = 0.5  # เพิ่มความเข้มสำหรับ left
    elif "right" in filename:
        adjustment_factor = 0.5  # ลดความเข้มสำหรับ right
    else:
        adjustment_factor = 1.0  # ค่าปกติสำหรับภาพอื่นๆ
    
    T = adaptive_threshold(gray_image, adjustment_factor=adjustment_factor)
    threshold_image = (gray_image > T).astype(np.uint8) * 255  # Generate threshold image
    
    save_path = os.path.join(output_path, f"processed_{filename}")  # Output path
    cv2.imwrite(save_path, threshold_image)  # Save image

print("การประมวลผลเสร็จสมบูรณ์")

พบ 236 ไฟล์ที่ตรงกับรูปแบบ
การประมวลผลเสร็จสมบูรณ์
